# 2.6 Single Pair Training with Full Eval

Based on 2.5, with improvements:
1. **25 epochs max** - prevent overfitting
2. **Full-pair evaluation** - eval on ALL pairs for stable signal
3. **Baseline comparison** - compare to model_final.pt
4. **Track strategy metrics** - IC Sharpe and top-K returns per epoch

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from dataclasses import dataclass

@dataclass
class ModelConfig:
    # Feature dimensions
    n_fundamental_features: int = 19
    n_price_features: int = 9
    n_embedding_dim: int = 768
    
    # Encoder latent dimensions
    fundamental_latent: int = 32
    price_latent: int = 16
    news_latent: int = 32
    
    # Dropout - tunable per encoder
    fundamental_dropout: float = 0.95  # HIGH - combat 60x overexposure
    price_dropout: float = 0.4         # MEDIUM
    news_dropout: float = 0.2          # Standard
    
    # News influence cap
    news_alpha: float = 0.8
    
    # Training
    batch_size: int = 512
    learning_rate: float = 1e-3
    weight_decay: float = 1e-3
    n_epochs: int = 25  # Reduced from 100

## 1. Load Data

In [ ]:
df = pd.read_parquet("data/ml_dataset.pqt")
df["feature_date"] = pd.to_datetime(df["feature_date"])

print(f"Dataset: {len(df):,} rows")
print(f"Date range: {df['feature_date'].min().date()} to {df['feature_date'].max().date()}")
print(f"Symbols: {df['symbol'].nunique():,}")

In [ ]:
# Feature columns
price_feat_cols = [
    "overnight_gap_z", "intraday_ret_z",
    "ret_1d_z", "ret_2d_z", "ret_3d_z", "ret_5d_z",
    "vol_5d_z", "dist_from_high_5d_z", "dist_from_low_5d_z"
]
fund_feat_cols = [c for c in df.columns if c.endswith("_z") and c not in price_feat_cols and c != "news_count_z"]
emb_cols = [c for c in df.columns if c.startswith("emb_")]

print(f"Price features: {len(price_feat_cols)}")
print(f"Fundamental features: {len(fund_feat_cols)}")
print(f"Embedding dims: {len(emb_cols)}")

In [ ]:
# Time-based split
dates = sorted(df["feature_date"].unique())
n_dates = len(dates)
train_end_idx = int(n_dates * 0.7)
val_end_idx = int(n_dates * 0.8)

train_dates = set(dates[:train_end_idx])
val_dates = set(dates[train_end_idx:val_end_idx])
test_dates = set(dates[val_end_idx:])

train_df = df[df["feature_date"].isin(train_dates)].copy()
val_df = df[df["feature_date"].isin(val_dates)].copy()
test_df = df[df["feature_date"].isin(test_dates)].copy()

print(f"Train: {len(train_df):,} rows, {len(train_dates)} days")
print(f"Val: {len(val_df):,} rows, {len(val_dates)} days")
print(f"Test: {len(test_df):,} rows, {len(test_dates)} days")

## 2. Dataset Classes

In [ ]:
class SinglePairDataset(Dataset):
    """Dataset where each symbol appears in exactly one pair per day.
    
    Shuffle symbols each day, pair consecutively: (0,1), (2,3), (4,5)...
    This ensures no symbol dominates training through repeated exposure.
    """

    def __init__(
        self,
        df: pd.DataFrame,
        price_cols: list[str],
        fund_cols: list[str],
        emb_cols: list[str],
    ):
        # Filter to rows with news only
        has_news = (df[emb_cols].abs().sum(axis=1) > 0)
        df_news = df[has_news].copy().reset_index(drop=True)
        print(f"Filtered to news-only: {len(df_news):,} / {len(df):,} rows ({len(df_news)/len(df)*100:.1f}%)")

        self.df = df_news
        self.price_cols = price_cols
        self.fund_cols = fund_cols
        self.emb_cols = emb_cols

        # Group indices by date - use reset df so indices match arrays
        self.date_groups = {}
        for date, group in self.df.groupby("feature_date"):
            indices = group.index.tolist()
            if len(indices) < 2:
                continue
            self.date_groups[date] = indices

        self.dates = list(self.date_groups.keys())
        print(f"Days with sufficient news coverage: {len(self.dates)}")

        # Precompute arrays (indices now 0 to len-1)
        self.price_arr = self.df[price_cols].values.astype(np.float32)
        self.fund_arr = self.df[fund_cols].values.astype(np.float32)
        self.emb_arr = self.df[emb_cols].values.astype(np.float32)
        self.target_arr = self.df["target_return"].values.astype(np.float32)

        # Generate initial pairs
        self.pairs = []
        self._generate_pairs()

    def _generate_pairs(self):
        """Generate pairs: shuffle each day, pair consecutively."""
        pairs = []
        
        for date in self.dates:
            indices = list(self.date_groups[date])  # copy
            np.random.shuffle(indices)
            
            # Pair consecutively: (0,1), (2,3), (4,5)...
            for i in range(0, len(indices) - 1, 2):
                pairs.append((indices[i], indices[i + 1]))
        
        self.pairs = pairs
        
        # Stats
        avg_pairs_per_day = len(pairs) / len(self.dates)
        print(f"Generated {len(self.pairs):,} pairs ({avg_pairs_per_day:.0f} per day avg)")

    def resample_pairs(self):
        """Regenerate pairs with new shuffle."""
        self._generate_pairs()

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        i, j = self.pairs[idx]

        price_i = self.price_arr[i]
        price_j = self.price_arr[j]
        fund_i = self.fund_arr[i]
        fund_j = self.fund_arr[j]
        emb_i = self.emb_arr[i]
        emb_j = self.emb_arr[j]

        actual_label = 1.0 if self.target_arr[i] > self.target_arr[j] else 0.0

        # Random swap for label balance
        if np.random.random() < 0.5:
            price_i, price_j = price_j, price_i
            fund_i, fund_j = fund_j, fund_i
            emb_i, emb_j = emb_j, emb_i
            label = 1.0 - actual_label
        else:
            label = actual_label

        return {
            "price_i": torch.tensor(price_i),
            "price_j": torch.tensor(price_j),
            "fund_i": torch.tensor(fund_i),
            "fund_j": torch.tensor(fund_j),
            "emb_i": torch.tensor(emb_i),
            "emb_j": torch.tensor(emb_j),
            "label": torch.tensor(label),
        }

## 3. Model Architecture

In [ ]:
class MultiBranchRanker(nn.Module):
    """Multi-branch model with high fundamental dropout."""
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        
        # High dropout for fundamentals
        self.fund_encoder = nn.Sequential(
            nn.Linear(config.n_fundamental_features, 64),
            nn.ReLU(),
            nn.Dropout(config.fundamental_dropout),
            nn.Linear(64, config.fundamental_latent),
            nn.ReLU(),
        )
        
        # Medium dropout for price
        self.price_encoder = nn.Sequential(
            nn.Linear(config.n_price_features, 32),
            nn.ReLU(),
            nn.Dropout(config.price_dropout),
            nn.Linear(32, config.price_latent),
            nn.ReLU(),
        )
        
        # Standard dropout for news
        self.news_encoder = nn.Sequential(
            nn.Linear(config.n_embedding_dim, 128),
            nn.ReLU(),
            nn.Dropout(config.news_dropout),
            nn.Linear(128, config.news_latent),
            nn.ReLU(),
        )
        
        fused_dim = config.fundamental_latent + config.price_latent + config.news_latent
        self.output_head = nn.Sequential(
            nn.Linear(fused_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(32, 1),
        )
    
    def forward(self, price, fund, emb):
        h_f = self.fund_encoder(fund)
        h_p = self.price_encoder(price)
        h_n = self.news_encoder(emb)
        h_n_scaled = self.config.news_alpha * h_n
        h = torch.cat([h_f, h_p, h_n_scaled], dim=-1)
        return self.output_head(h).squeeze(-1)
    
    def forward_pair(self, price_i, fund_i, emb_i, price_j, fund_j, emb_j):
        score_i = self.forward(price_i, fund_i, emb_i)
        score_j = self.forward(price_j, fund_j, emb_j)
        return torch.sigmoid(score_i - score_j)

## 4. Training and Evaluation Functions

In [ ]:
def pairwise_ranking_loss(pred_prob, label, smoothing=0.1):
    smoothed_label = label * (1 - smoothing) + 0.5 * smoothing
    return F.binary_cross_entropy(pred_prob, smoothed_label)


def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    for batch in tqdm(loader, desc="Training", leave=False):
        price_i = batch["price_i"].to(device)
        price_j = batch["price_j"].to(device)
        fund_i = batch["fund_i"].to(device)
        fund_j = batch["fund_j"].to(device)
        emb_i = batch["emb_i"].to(device)
        emb_j = batch["emb_j"].to(device)
        label = batch["label"].to(device)
        
        optimizer.zero_grad()
        pred_prob = model.forward_pair(price_i, fund_i, emb_i, price_j, fund_j, emb_j)
        loss = pairwise_ranking_loss(pred_prob, label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * len(label)
        total_correct += ((pred_prob > 0.5) == (label > 0.5)).sum().item()
        total_samples += len(label)
    
    return total_loss / total_samples, total_correct / total_samples


@torch.no_grad()
def get_scores(model, df, price_cols, fund_cols, emb_cols, device, batch_size=1024):
    """Score all rows in dataframe."""
    model.eval()
    
    price_arr = torch.tensor(df[price_cols].values.astype(np.float32))
    fund_arr = torch.tensor(df[fund_cols].values.astype(np.float32))
    emb_arr = torch.tensor(df[emb_cols].values.astype(np.float32))
    
    scores = []
    for i in range(0, len(df), batch_size):
        price = price_arr[i:i+batch_size].to(device)
        fund = fund_arr[i:i+batch_size].to(device)
        emb = emb_arr[i:i+batch_size].to(device)
        score = model(price, fund, emb)
        scores.append(score.cpu().numpy())
    
    return np.concatenate(scores)


def compute_daily_ic(df):
    """Compute Spearman IC per day."""
    ics = []
    for date, group in df.groupby("feature_date"):
        if len(group) < 10:
            continue
        ic, _ = spearmanr(group["score"], group["target_return"])
        if not np.isnan(ic):
            ics.append({"date": date, "ic": ic})
    return pd.DataFrame(ics)


def compute_short_returns(df, k=5, clip_return=0.10):
    """Compute daily short returns for bottom-K with return clipping."""
    returns = []
    for date, group in df.groupby("feature_date"):
        if len(group) < k * 2:
            continue
        bottom = group.nsmallest(k, "score")
        clipped_returns = bottom["target_return"].clip(-clip_return, clip_return)
        short_ret = -clipped_returns.mean()
        returns.append({"date": date, "return": short_ret})
    return pd.DataFrame(returns)


def evaluate_model(model, df, price_cols, fund_cols, emb_cols, device, k=5):
    """Full evaluation: IC Sharpe and strategy metrics."""
    df_eval = df.copy()
    df_eval["score"] = get_scores(model, df_eval, price_cols, fund_cols, emb_cols, device)
    
    # IC metrics
    ic_df = compute_daily_ic(df_eval)
    mean_ic = ic_df['ic'].mean()
    ic_std = ic_df['ic'].std()
    ic_sharpe = mean_ic / ic_std * np.sqrt(252) if ic_std > 0 else 0
    
    # Strategy metrics
    short_df = compute_short_returns(df_eval, k=k, clip_return=0.10)
    if len(short_df) > 1:
        short_mean = short_df['return'].mean()
        short_std = short_df['return'].std()
        short_sharpe = short_mean / short_std * np.sqrt(252) if short_std > 0 else 0
        short_cumret = (1 + short_df['return']).cumprod().iloc[-1] - 1
    else:
        short_sharpe = 0
        short_cumret = 0
    
    return {
        "mean_ic": mean_ic,
        "ic_sharpe": ic_sharpe,
        "short_sharpe": short_sharpe,
        "short_cumret": short_cumret,
    }

## 5. Load Baseline Model

In [ ]:
# Load baseline model_final.pt for comparison
baseline_path = Path("data/model_final.pt")
if baseline_path.exists():
    baseline_ckpt = torch.load(baseline_path, map_location=device, weights_only=False)
    
    # Create baseline model with same config structure
    baseline_config = baseline_ckpt.get("config", ModelConfig())
    baseline_model = MultiBranchRanker(baseline_config).to(device)
    baseline_model.load_state_dict(baseline_ckpt["model_state_dict"])
    baseline_model.eval()
    
    print("Loaded baseline model_final.pt")
    print(f"  Fundamental dropout: {baseline_config.fundamental_dropout}")
    print(f"  Price dropout: {baseline_config.price_dropout}")
else:
    baseline_model = None
    print("No baseline model found")

In [ ]:
# Evaluate baseline on val and test sets
if baseline_model is not None:
    print("Evaluating baseline model...")
    baseline_val = evaluate_model(baseline_model, val_df, price_feat_cols, fund_feat_cols, emb_cols, device)
    baseline_test = evaluate_model(baseline_model, test_df, price_feat_cols, fund_feat_cols, emb_cols, device)
    
    print("\nBaseline Results:")
    print(f"  Val  IC Sharpe: {baseline_val['ic_sharpe']:.2f}, Short Sharpe: {baseline_val['short_sharpe']:.2f}")
    print(f"  Test IC Sharpe: {baseline_test['ic_sharpe']:.2f}, Short Sharpe: {baseline_test['short_sharpe']:.2f}")
else:
    baseline_val = baseline_test = None

## 6. Train Model

In [ ]:
# Configuration
config = ModelConfig(
    n_fundamental_features=len(fund_feat_cols),
    n_price_features=len(price_feat_cols),
    n_embedding_dim=len(emb_cols),
    fundamental_dropout=0.95,  # HIGH
    price_dropout=0.4,         # MEDIUM
    news_dropout=0.2,          # Standard
    n_epochs=25,               # Reduced
)

print(f"Config:")
print(f"  Epochs: {config.n_epochs}")
print(f"  Fundamental dropout: {config.fundamental_dropout}")
print(f"  Price dropout: {config.price_dropout}")
print(f"  News dropout: {config.news_dropout}")

In [ ]:
# Create training dataset (single-pair)
train_dataset = SinglePairDataset(train_df, price_feat_cols, fund_feat_cols, emb_cols)
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

In [ ]:
# Create model
model = MultiBranchRanker(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.n_epochs)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Train with full evaluation each epoch
best_val_ic_sharpe = -float('inf')
best_state = None
history = []

for epoch in range(config.n_epochs):
    # Reshuffle pairs each epoch
    train_dataset.resample_pairs()
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
    scheduler.step()
    
    # Full evaluation on val set (ALL rows, not single-pair)
    val_metrics = evaluate_model(model, val_df, price_feat_cols, fund_feat_cols, emb_cols, device)
    
    history.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_ic_sharpe": val_metrics["ic_sharpe"],
        "val_short_sharpe": val_metrics["short_sharpe"],
        "val_mean_ic": val_metrics["mean_ic"],
    })
    
    print(f"Epoch {epoch+1}/{config.n_epochs}: "
          f"loss={train_loss:.4f}, acc={train_acc:.4f}, "
          f"val_IC_sharpe={val_metrics['ic_sharpe']:.2f}, "
          f"val_short_sharpe={val_metrics['short_sharpe']:.2f}")
    
    # Track best by IC Sharpe
    if val_metrics["ic_sharpe"] > best_val_ic_sharpe:
        best_val_ic_sharpe = val_metrics["ic_sharpe"]
        best_state = model.state_dict().copy()

# Load best model
model.load_state_dict(best_state)
print(f"\nBest val IC Sharpe: {best_val_ic_sharpe:.2f}")

In [ ]:
# Plot training curves
hist_df = pd.DataFrame(history)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Loss
ax = axes[0, 0]
ax.plot(hist_df['epoch'], hist_df['train_loss'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Loss')
ax.grid(True, alpha=0.3)

# Accuracy
ax = axes[0, 1]
ax.plot(hist_df['epoch'], hist_df['train_acc'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Training Accuracy')
ax.grid(True, alpha=0.3)

# Val IC Sharpe
ax = axes[1, 0]
ax.plot(hist_df['epoch'], hist_df['val_ic_sharpe'], label='Single-pair model')
if baseline_val is not None:
    ax.axhline(baseline_val['ic_sharpe'], color='red', linestyle='--', label=f'Baseline: {baseline_val["ic_sharpe"]:.2f}')
ax.set_xlabel('Epoch')
ax.set_ylabel('IC Sharpe')
ax.set_title('Validation IC Sharpe')
ax.legend()
ax.grid(True, alpha=0.3)

# Val Short Sharpe
ax = axes[1, 1]
ax.plot(hist_df['epoch'], hist_df['val_short_sharpe'], label='Single-pair model')
if baseline_val is not None:
    ax.axhline(baseline_val['short_sharpe'], color='red', linestyle='--', label=f'Baseline: {baseline_val["short_sharpe"]:.2f}')
ax.set_xlabel('Epoch')
ax.set_ylabel('Short Sharpe')
ax.set_title('Validation Short Strategy Sharpe (K=5)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Final Test Evaluation

In [ ]:
# Final evaluation on test set
test_metrics = evaluate_model(model, test_df, price_feat_cols, fund_feat_cols, emb_cols, device)

print("=" * 60)
print("TEST SET RESULTS")
print("=" * 60)
print(f"Mean IC:       {test_metrics['mean_ic']:.4f}")
print(f"IC Sharpe:     {test_metrics['ic_sharpe']:.2f}")
print(f"Short Sharpe:  {test_metrics['short_sharpe']:.2f}")
print(f"Short Return:  {test_metrics['short_cumret']*100:.1f}%")

if baseline_test is not None:
    print("\n" + "-" * 60)
    print("BASELINE COMPARISON (model_final.pt)")
    print("-" * 60)
    print(f"Baseline IC Sharpe:     {baseline_test['ic_sharpe']:.2f}")
    print(f"Baseline Short Sharpe:  {baseline_test['short_sharpe']:.2f}")
    print(f"\nImprovement:")
    print(f"  IC Sharpe:    {test_metrics['ic_sharpe'] - baseline_test['ic_sharpe']:+.2f}")
    print(f"  Short Sharpe: {test_metrics['short_sharpe'] - baseline_test['short_sharpe']:+.2f}")

In [ ]:
# Detailed IC analysis
test_df_eval = test_df.copy()
test_df_eval["score"] = get_scores(model, test_df_eval, price_feat_cols, fund_feat_cols, emb_cols, device)

ic_df = compute_daily_ic(test_df_eval)
short_df = compute_short_returns(test_df_eval, k=5, clip_return=0.10)

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Daily IC
ax = axes[0]
ax.bar(ic_df['date'], ic_df['ic'], alpha=0.7, width=1)
ax.axhline(0, color='black', linewidth=0.5)
ax.axhline(ic_df['ic'].mean(), color='red', linestyle='--', 
           label=f'Mean IC: {ic_df["ic"].mean():.4f}')
ax.set_ylabel('Daily IC')
ax.set_title('Information Coefficient Over Time')
ax.legend()

# Cumulative returns
ax = axes[1]
cumret = (1 + short_df['return']).cumprod()
ax.plot(short_df['date'], cumret, label=f'Single-pair model (Sharpe: {test_metrics["short_sharpe"]:.2f})')
ax.axhline(1, color='black', linestyle='--', alpha=0.5)
ax.set_ylabel('Cumulative Return')
ax.set_title('Short Strategy (K=5)')
ax.legend()

plt.tight_layout()
plt.show()

## 8. Save Model

In [ ]:
torch.save({
    "model_state_dict": model.state_dict(),
    "config": config,
    "price_cols": price_feat_cols,
    "fund_cols": fund_feat_cols,
    "emb_cols": emb_cols,
    "training_approach": "single_pair_per_symbol_v2",
    "results": {
        "test_mean_ic": test_metrics["mean_ic"],
        "test_ic_sharpe": test_metrics["ic_sharpe"],
        "test_short_sharpe": test_metrics["short_sharpe"],
    },
    "baseline_comparison": baseline_test,
}, "data/model_single_pair_v2.pt")

print("Saved model to data/model_single_pair_v2.pt")